In [1]:
import kagglehub
import os
import pandas as pd

# Download datasets
path_recommendation = kagglehub.dataset_download("arashnic/book-recommendation-dataset")
path_amazon = kagglehub.dataset_download("mohamedbakhet/amazon-books-reviews")
path_goodreads = kagglehub.dataset_download("jealousleopard/goodreadsbooks")

# Function to list files in each path
def list_files(path, name):
    print(f"\nFiles in {name}:")
    files = os.listdir(path)
    for file in files:
        print(f"- {file}")
    return files

# List files to confirm
list_files(path_recommendation, "Book Recommendation Dataset")
list_files(path_amazon, "Amazon Books Reviews")
list_files(path_goodreads, "Goodreads Books")

c:\Users\cmoan\miniforge3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Files in Book Recommendation Dataset:
- Books.csv
- classicRec.png
- DeepRec.png
- Ratings.csv
- recsys_taxonomy2.png
- Users.csv

Files in Amazon Books Reviews:
- books_data.csv
- Books_rating.csv

Files in Goodreads Books:
- books.csv


['books.csv']

BOOKS RECOMENDATIONS

BOOKS.CSV CLEAN

In [2]:
import pandas as pd
import os
import kagglehub

# Ensure the path is defined
path_recommendation = kagglehub.dataset_download("arashnic/book-recommendation-dataset")

# Define the file path and load
books_path = os.path.join(path_recommendation, 'Books.csv')
df_books = pd.read_csv(books_path, low_memory=False)

# Initial column normalization (lowercase, no spaces, no dots, no hyphens)
df_books.columns = df_books.columns.str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace('.', '_')

# Diagnostic: Info, Nulls and Data Types
print("--- Info & Column Types ---")
df_books.info()

print("\n--- Count of Null Values ---")
print(df_books.isnull().sum())

print("\n--- Unique values in 'year_of_publication' (Sample) ---")
print(df_books['year_of_publication'].unique()[:15])

print(f"\nTotal Rows: {len(df_books)}")

--- Info & Column Types ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271360 non-null  object
 1   book_title           271360 non-null  object
 2   book_author          271358 non-null  object
 3   year_of_publication  271360 non-null  object
 4   publisher            271358 non-null  object
 5   image_url_s          271360 non-null  object
 6   image_url_m          271360 non-null  object
 7   image_url_l          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB

--- Count of Null Values ---
isbn                   0
book_title             0
book_author            2
year_of_publication    0
publisher              2
image_url_s            0
image_url_m            0
image_url_l            3
dtype: int64

--- Unique values in 'year_of_publication' (Sample) ---
['2002' '2001' '1991'

In [3]:
# Filling Nulls
df_books['book_author'] = df_books['book_author'].fillna('unknown')
df_books['publisher'] = df_books['publisher'].fillna('unknown')
df_books['image_url_l'] = df_books['image_url_l'].fillna('no_url')

# Convert Year to Numeric (using errors='coerce' to turn strings into NaN)
df_books['year_of_publication'] = pd.to_numeric(df_books['year_of_publication'], errors='coerce')

# Text Normalization: Lowercase and strip spaces for all string columns
string_cols = ['book_title', 'book_author', 'publisher']
for col in string_cols:
    df_books[col] = df_books[col].astype(str).str.lower().str.strip()

# Check results
print("--- Final Check for Books.csv ---")
print(df_books.isnull().sum())
print(f"\nYear Dtype: {df_books['year_of_publication'].dtype}")

print(df_books[['book_title', 'book_author']].head())

--- Final Check for Books.csv ---
isbn                   0
book_title             0
book_author            0
year_of_publication    3
publisher              0
image_url_s            0
image_url_m            0
image_url_l            0
dtype: int64

Year Dtype: float64
                                          book_title           book_author
0                                classical mythology    mark p. o. morford
1                                       clara callan  richard bruce wright
2                               decision in normandy          carlo d'este
3  flu: the story of the great influenza pandemic...      gina bari kolata
4                             the mummies of urumchi       e. j. w. barber


In [4]:
# Drop unnecessary image columns
cols_to_drop = ['image_url_s', 'image_url_m', 'image_url_l']
df_books = df_books.drop(columns=cols_to_drop)

# Remove rows with any null values
# This will clean the remaining nulls in author, publisher and potential NaNs in year
df_books = df_books.dropna()

# Final verification of the clean dataframe
print("--- Final Columns ---")
print(df_books.columns.tolist())
print("\n--- Remaining Nulls ---")
print(df_books.isnull().sum())
print(f"\nTotal rows after cleaning: {len(df_books)}")
print("\n--- Preview ---")
print(df_books.head())

--- Final Columns ---
['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher']

--- Remaining Nulls ---
isbn                   0
book_title             0
book_author            0
year_of_publication    0
publisher              0
dtype: int64

Total rows after cleaning: 271357

--- Preview ---
         isbn                                         book_title  \
0  0195153448                                classical mythology   
1  0002005018                                       clara callan   
2  0060973129                               decision in normandy   
3  0374157065  flu: the story of the great influenza pandemic...   
4  0393045218                             the mummies of urumchi   

            book_author  year_of_publication                   publisher  
0    mark p. o. morford               2002.0     oxford university press  
1  richard bruce wright               2001.0       harperflamingo canada  
2          carlo d'este               1991.0           

RATING CLEAN

In [5]:
# Load the Ratings file
import re

ratings_path = os.path.join(path_recommendation, 'Ratings.csv')
df_ratings = pd.read_csv(ratings_path, low_memory=False)

# Normalize column names
df_ratings.columns = df_ratings.columns.str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace('.', '_')

# Initial Diagnostic
print("--- Ratings Info ---")
print(df_ratings.info())
print("\n--- Nulls in Ratings ---")
print(df_ratings.isnull().sum())

# Standardizing ISBN and Text
# It is crucial that ISBN matches the format in df_books
df_ratings['isbn'] = df_ratings['isbn'].astype(str).str.lower()
df_ratings['isbn'] = df_ratings['isbn'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df_ratings['isbn'] = df_ratings['isbn'].str.strip()

# 5.5 Check Rating distribution (optional but useful)
print("\n--- Rating Value Counts (Sample) ---")
print(df_ratings['book_rating'].value_counts().head())

--- Ratings Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   user_id      1149780 non-null  int64 
 1   isbn         1149780 non-null  object
 2   book_rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
None

--- Nulls in Ratings ---
user_id        0
isbn           0
book_rating    0
dtype: int64

--- Rating Value Counts (Sample) ---
book_rating
0     716109
8     103736
10     78610
7      76457
9      67541
Name: count, dtype: int64


USERS CLEAN

In [6]:
# Load the raw file
users_path = os.path.join(path_recommendation, 'Users.csv')
df_users = pd.read_csv(users_path, low_memory=False)

# Standardize column names first (to work comfortably)
df_users.columns = df_users.columns.str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace('.', '_')

# Diagnostic: Types and Nulls
print("--- Info & Types ---")
print(df_users.info())

print("\n--- Count of Nulls ---")
print(df_users.isnull().sum())

# Statistical summary of Age
print("\n--- Age Column Statistics ---")
print(df_users['age'].describe())

# Sample of Location strings
print("\n--- Sample of Location Data ---")
print(df_users['location'].head(15))

--- Info & Types ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   user_id   278858 non-null  int64  
 1   location  278858 non-null  object 
 2   age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB
None

--- Count of Nulls ---
user_id          0
location         0
age         110762
dtype: int64

--- Age Column Statistics ---
count    168096.000000
mean         34.751434
std          14.428097
min           0.000000
25%          24.000000
50%          32.000000
75%          44.000000
max         244.000000
Name: age, dtype: float64

--- Sample of Location Data ---
0                     nyc, new york, usa
1              stockton, california, usa
2        moscow, yukon territory, russia
3              porto, v.n.gaia, portugal
4     farnborough, hants, united kingdom
5          santa monica, cali

In [7]:
import numpy as np

# Cleaning Age: Replace outliers (under 5 and over 95) with NaN
df_users.loc[(df_users['age'] > 95) | (df_users['age'] < 5), 'age'] = np.nan

# Fill NaNs with the median (more robust than mean)
median_age = df_users['age'].median()
df_users['age'] = df_users['age'].fillna(median_age).astype(int)

# Split Location into City, State, Country
# We split by comma, and take only the first 3 parts if available
loc_split = df_users['location'].str.split(',', n=2, expand=True)
df_users['city'] = loc_split[0]
df_users['state'] = loc_split[1]
df_users['country'] = loc_split[2]

# Normalize and Clean the new columns
for col in ['city', 'state', 'country']:
    df_users[col] = df_users[col].astype(str).str.lower()
    # Remove special characters except spaces
    df_users[col] = df_users[col].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    df_users[col] = df_users[col].str.strip()
    # Handle empty results from the split
    df_users.loc[df_users[col] == '', col] = 'unknown'

#  Drop original location column
df_users = df_users.drop(columns=['location'])

# Final check
print("--- Final Age Stats ---")
print(df_users['age'].describe())
print("\n--- Cleaned Users Sample ---")
print(df_users.head())

--- Final Age Stats ---
count    278858.000000
mean         33.638856
std          10.617503
min           5.000000
25%          29.000000
50%          32.000000
75%          35.000000
max          95.000000
Name: age, dtype: float64

--- Cleaned Users Sample ---
   user_id  age         city            state         country
0        1   32          nyc         new york             usa
1        2   18     stockton       california             usa
2        3   32       moscow  yukon territory          russia
3        4   17        porto           vngaia        portugal
4        5   32  farnborough            hants  united kingdom


In [8]:
#  Count unique values
unique_cities = df_users['city'].nunique()
unique_countries = df_users['country'].nunique()

print(f"Total unique cities: {unique_cities}")
print(f"Total unique countries: {unique_countries}")

# Top 20 Countries by user count
print("\n--- Top 20 Countries ---")
print(df_users['country'].value_counts().head(1120))

# Top 20 Cities by user count
print("\n--- Top 20 Cities ---")
print(df_users['city'].value_counts().head(30))

# Check for very short country names (potential errors)
print("\n--- Potential Country Errors (too short) ---")
print(df_users[df_users['country'].str.len() <= 2]['country'].unique())

Total unique cities: 32381
Total unique countries: 1120

--- Top 20 Countries ---
country
usa                        139190
canada                      21557
united kingdom              18311
germany                     17051
spain                       13198
                            ...  
prchina china                   1
hebei china                     1
laoning china                   1
estado de mexico mexico         1
piemonte pakistan               1
Name: count, Length: 1120, dtype: int64

--- Top 20 Cities ---
city
london           4107
barcelona        2665
toronto          2342
madrid           1934
sydney           1885
portland         1869
melbourne        1863
vancouver        1699
chicago          1566
seattle          1541
new york         1445
milano           1390
san diego        1331
berlin           1306
ottawa           1303
san francisco    1300
houston          1241
paris            1191
roma             1045
austin           1044
los angeles      1042
calgar

In [9]:
# Dictionary for major country corrections
country_map = {
    'usa': 'usa',
    'united states': 'usa',
    'united kingdom': 'uk',
    'england': 'uk',
    'scotland': 'uk',
    'wales': 'uk',
    'china': 'china',
    'mexico': 'mexico',
    'spain': 'spain',
    'españa': 'spain',
    'canada': 'canada',
    'germany': 'germany',
    'deutschland': 'germany',
    'australia': 'australia',
    'italy': 'italy',
    'italia': 'italy',
    'france': 'france',
    'portugal': 'portugal',
    'brazil': 'brazil',
    'brasil': 'brazil',
    'india': 'india'
}

def clean_country_advanced(country):
    country = str(country).lower()
    # Check if any key from our map is inside the string
    for key, value in country_map.items():
        if key in country:
            return value
    return country # Return as is if no match found

# Apply the cleaning function
df_users['country'] = df_users['country'].apply(clean_country_advanced)

# Filter: Keep only countries with a minimum number of users to avoid "noise"
# For an expansion strategy, countries with 1 or 2 users are not statistically significant
country_counts = df_users['country'].value_counts()
major_countries = country_counts[country_counts >= 10].index
df_users.loc[~df_users['country'].isin(major_countries), 'country'] = 'other'

print("--- Top 125 Cleaned Countries ---")
print(df_users['country'].value_counts().head(125))

--- Top 125 Cleaned Countries ---
country
usa          139741
canada        21662
uk            18660
germany       17102
spain         13323
              ...  
catalunya        13
benin            13
fiji             12
syria            12
ethiopia         12
Name: count, Length: 125, dtype: int64


In [10]:
# Count unique values after cleaning
final_unique_cities = df_users['city'].nunique()
final_unique_countries = df_users['country'].nunique()

print(f"Total unique cities after cleaning: {final_unique_cities}")
print(f"Total unique countries after cleaning: {final_unique_countries}")

# Calculate the coverage of our cleaning
other_count = (df_users['country'] == 'other').sum()
total_users = len(df_users)
coverage_pct = ((total_users - other_count) / total_users) * 100

print(f"\nUsers in major countries: {total_users - other_count} ({coverage_pct:.2f}%)")
print(f"Users classified as 'other': {other_count} ({100 - coverage_pct:.2f}%)")

Total unique cities after cleaning: 32381
Total unique countries after cleaning: 137

Users in major countries: 277843 (99.64%)
Users classified as 'other': 1015 (0.36%)


In [11]:
# Specific mapping for regions and duplicates
manual_map = {
    'catalunya': 'spain',
    'euskal herria': 'spain',
    'united arab emirates': 'uae',
    'urugua': 'uruguay',
    'na philippines': 'philippines',
    'na': 'unknown'
}

df_users['country'] = df_users['country'].replace(manual_map)

# Smart Imputation based on City/State
# We define a small dictionary of common cities to rescue 'unknown' countries
city_to_country = {
    'madrid': 'spain',
    'barcelona': 'spain',
    'london': 'uk',
    'nyc': 'usa',
    'new york': 'usa',
    'toronto': 'canada',
    'paris': 'france',
    'lisboa': 'portugal',
    'milan': 'italy'
}

def rescue_country(row):
    if row['country'] in ['unknown', 'other', 'na']:
        # Check city
        if row['city'] in city_to_country:
            return city_to_country[row['city']]
        # Check state
        if row['state'] in city_to_country:
            return city_to_country[row['state']]
    return row['country']

df_users['country'] = df_users.apply(rescue_country, axis=1)

# Final cleanup: replace anything still looking like 'na' with 'unknown'
df_users['country'] = df_users['country'].replace(['nan', 'other'], 'unknown')


In [12]:
# Count occurrences of 'unknown'
unknown_count = (df_users['country'] == 'unknown').sum()

# Count actual NaN values (nulls)
nan_count = df_users['country'].isna().sum()

# Count empty strings (if any)
empty_count = (df_users['country'] == '').sum()

print(f"Resultados de la búsqueda:")
print(f"- Total de 'unknown': {unknown_count}")
print(f"- Total de NaNs (nulos): {nan_count}")
print(f"- Total de cadenas vacías: {empty_count}")

# Mostrar porcentaje de datos limpios vs desconocidos
total_users = len(df_users)
clean_pct = ((total_users - unknown_count - nan_count) / total_users) * 100
print(f"\nCobertura final de países válidos: {clean_pct:.2f}%")

Resultados de la búsqueda:
- Total de 'unknown': 5375
- Total de NaNs (nulos): 0
- Total de cadenas vacías: 0

Cobertura final de países válidos: 98.07%


In [13]:
# Filtering users who are 'unknown' in all three geographic fields
total_unknown = df_users[
    (df_users['city'] == 'unknown') & 
    (df_users['state'] == 'unknown') & 
    (df_users['country'] == 'unknown')
]

# Calculating counts
num_total_unknown = len(total_unknown)
num_partial_unknown = 5375 - num_total_unknown # 5375 was your previous count

print(f"--- Análisis de Calidad Geográfica ---")
print(f"Usuarios sin NINGÚN dato (Ciudad, Estado y País son 'unknown'): {num_total_unknown}")
print(f"Usuarios con 'unknown' en país pero con Ciudad o Estado válido: {num_partial_unknown}")

# Previewing those who still have some data
if num_partial_unknown > 0:
    print("\n--- Muestra de usuarios con datos parciales (rescatables) ---")
    print(df_users[(df_users['country'] == 'unknown') & (df_users['city'] != 'unknown')][['city', 'state', 'country']].head(10))

--- Análisis de Calidad Geográfica ---
Usuarios sin NINGÚN dato (Ciudad, Estado y País son 'unknown'): 65
Usuarios con 'unknown' en país pero con Ciudad o Estado válido: 5310

--- Muestra de usuarios con datos parciales (rescatables) ---
                 city        state  country
18             weston      unknown  unknown
42             méxico  méxico city  unknown
90   torontonewmarket      unknown  unknown
256              quit         quit  unknown
257               ljk          lkj  unknown
311                lk          jlk  unknown
317            berlin           na  unknown
536             davis   california  unknown
557           burnaby      unknown  unknown
571             gabes        gabes  unknown


In [14]:
# Remove users with NO geographic data at all
df_users = df_users.drop(df_users[(df_users['city'] == 'unknown') & 
                                  (df_users['state'] == 'unknown') & 
                                  (df_users['country'] == 'unknown')].index)

# Smart Rescue: If city or state contains a known country name
valid_countries = [c for c in df_users['country'].unique() if c != 'unknown']

def final_geo_rescue(row):
    if row['country'] == 'unknown':
        # Check if the city or state mentions a known country
        for country in valid_countries:
            if country in str(row['city']) or country in str(row['state']):
                return country
    return row['country']

df_users['country'] = df_users.apply(final_geo_rescue, axis=1)

# Final Cut: Remove any remaining 'unknown' countries 
# (These are the ones with gibberish like 'ljk' or 'quit')
df_users = df_users[df_users['country'] != 'unknown']

print(f"--- Dataset Finalizado ---")
print(f"Total de usuarios finales: {len(df_users)}")
print(f"¿Nulos o unknowns en Country?: {df_users['country'].isin(['unknown']).sum()}")
print(f"Número de países únicos finales: {df_users['country'].nunique()}")

--- Dataset Finalizado ---
Total de usuarios finales: 273756
¿Nulos o unknowns en Country?: 0
Número de países únicos finales: 129


In [15]:
df_users.head()

,user_id,age,city,state,country
0,1,32,nyc,new york,usa
1,2,18,stockton,california,usa
2,3,32,moscow,yukon territory,russia
3,4,17,porto,vngaia,portugal
4,5,32,farnborough,hants,uk


UNION

In [16]:
# Check types for ISBN
print(f"ISBN type in Books: {df_books['isbn'].dtype}")
print(f"ISBN type in Ratings: {df_ratings['isbn'].dtype}")

# Check types for User_ID
print(f"User_ID type in Users: {df_users['user_id'].dtype}")
print(f"User_ID type in Ratings: {df_ratings['user_id'].dtype}")

# Quick check: How many ISBNs in Ratings actually exist in Books?
common_isbns = df_ratings['isbn'].isin(df_books['isbn']).sum()
print(f"\nRatings with a matching book: {common_isbns} out of {len(df_ratings)}")

ISBN type in Books: object
ISBN type in Ratings: object
User_ID type in Users: int64
User_ID type in Ratings: int64

Ratings with a matching book: 959960 out of 1149780


In [17]:
# First merge: Ratings + Books (Key: isbn)
df_temp = pd.merge(df_ratings, df_books, on='isbn', how='inner')

# Second merge: (Ratings+Books) + Users (Key: user_id)
df_master = pd.merge(df_temp, df_users, on='user_id', how='inner')

# Verify dimensions
print(f"Total rows in Master Dataset: {len(df_master)}")
print(f"Total columns: {len(df_master.columns)}")

# Export to CSV (Optional but recommended)
# df_master.to_csv('Master_Book_Data.csv', index=False)

print("\n--- Master Dataset Head ---")
print(df_master.head())

Total rows in Master Dataset: 927222
Total columns: 11

--- Master Dataset Head ---
   user_id        isbn  book_rating  \
0   276726  0155061224            5   
1   276727  0446520802            0   
2   276729  0521795028            6   
3   276733  2080674722            0   
4   276747  0060517794            9   

                                          book_title         book_author  \
0                                   rites of passage          judith rae   
1                                       the notebook     nicholas sparks   
2  the amsterdam connection : level 4 (cambridge ...         sue leather   
3                        les particules elementaires  michel houellebecq   
4                           little altars everywhere       rebecca wells   

   year_of_publication                   publisher  age       city  \
0               2001.0                      heinle   32    seattle   
1               1996.0                warner books   16          h   
2             

DF GLOBAL BOOKS RECOMENDATION

In [18]:
df_master.info()
df_master.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 927222 entries, 0 to 927221
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              927222 non-null  int64  
 1   isbn                 927222 non-null  object 
 2   book_rating          927222 non-null  int64  
 3   book_title           927222 non-null  object 
 4   book_author          927222 non-null  object 
 5   year_of_publication  927222 non-null  float64
 6   publisher            927222 non-null  object 
 7   age                  927222 non-null  int64  
 8   city                 927222 non-null  object 
 9   state                927222 non-null  object 
 10  country              927222 non-null  object 
dtypes: float64(1), int64(3), object(7)
memory usage: 77.8+ MB


,user_id,isbn,book_rating,book_title,book_author,year_of_publication,publisher,age,city,state,country
0,276726,0155061224,5,rites of passage,judith rae,2001.0,heinle,32,seattle,washington,usa
1,276727,0446520802,0,the notebook,nicholas sparks,1996.0,warner books,16,h,new south wales,australia
2,276729,0521795028,6,the amsterdam connection : level 4 (cambridge ...,sue leather,2001.0,cambridge university press,16,rijeka,na,croatia
3,276733,2080674722,0,les particules elementaires,michel houellebecq,1998.0,flammarion,37,paris,na,france
4,276747,0060517794,9,little altars everywhere,rebecca wells,2003.0,harpertorch,25,iowa city,iowa,usa


In [19]:
## Export Master DataFrame to CSV

import pandas as pd

# We use 'index=False' to prevent pandas from creating an extra column for the row numbers.
# We use 'encoding="utf-8"' to ensure special characters (like accents in book titles) are saved correctly.
df_master.to_csv('Master_RecomendationBook_Data.csv', index=False, encoding='utf-8')

print("Process Complete: 'Master_RecomendationBook_Data.csv' has been created.")

Process Complete: 'Master_RecomendationBook_Data.csv' has been created.


AMAZON

BOOKS CLEAN

In [20]:
import pandas as pd
import os

# Load the file
amazon_books_path = os.path.join(path_amazon, 'books_data.csv')
df_amazon_books = pd.read_csv(amazon_books_path)

# Basic Info: Data types and memory usage
print("--- 1. Data Types and Info ---")
print(df_amazon_books.info())

# Detailed Null Analysis (Count and Percentage)
null_counts = df_amazon_books.isnull().sum()
null_percentages = (null_counts / len(df_amazon_books)) * 100
null_report = pd.DataFrame({'Null Count': null_counts, 'Percentage (%)': null_percentages})

print("\n--- 2. Null Values Report ---")
print(null_report.sort_values(by='Percentage (%)', ascending=False))

# Statistical Summary of Numeric Columns
print("\n--- 3. Statistical Summary (Numeric) ---")
print(df_amazon_books.describe())

# Inspecting Object Columns (Categories, Price, Title)
print("\n--- 4. Sample of Categorical/Text Columns ---")
print(df_amazon_books.head(10))

--- 1. Data Types and Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Title          212403 non-null  object 
 1   description    143962 non-null  object 
 2   authors        180991 non-null  object 
 3   image          160329 non-null  object 
 4   previewLink    188568 non-null  object 
 5   publisher      136518 non-null  object 
 6   publishedDate  187099 non-null  object 
 7   infoLink       188568 non-null  object 
 8   categories     171205 non-null  object 
 9   ratingsCount   49752 non-null   float64
dtypes: float64(1), object(9)
memory usage: 16.2+ MB
None

--- 2. Null Values Report ---
               Null Count  Percentage (%)
ratingsCount       162652       76.576712
publisher           75886       35.727199
description         68442       32.222557
image               52075       24.516958
categories         

In [21]:
import re

# List of columns for aggressive cleaning (Excluding publishedDate)
text_cols = ['Title', 'description', 'authors', 'publisher', 'categories']

# Redefining deep clean for text only
def deep_clean_text(text):
    if pd.isna(text) or str(text).lower() == 'nan':
        return 'unknown'
    text = str(text).lower()
    # Removes everything except letters, numbers and spaces
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return " ".join(text.split()).strip()

# Apply aggressive cleaning to text columns
for col in text_cols:
    if col in df_amazon_books.columns:
        df_amazon_books[col] = df_amazon_books[col].apply(deep_clean_text)

# Special handling for publishedDate (No regex, just basic cleanup)
df_amazon_books['publishedDate'] = df_amazon_books['publishedDate'].fillna('unknown').astype(str).str.strip()

# Verification
print("--- Amazon Data: Selective Cleaning Results ---")
print(df_amazon_books.head(10))

--- Amazon Data: Selective Cleaning Results ---
                                               Title  \
0                      its only art if its well hung   
1                             dr seuss american icon   
2              wonderful worship in smaller churches   
3                      whispers of the wicked saints   
4  nation dance religion identity and cultural di...   
5  the church of christ a biblical ecclesiology f...   
6                           the overbury affair avon   
7             a walk in the woods a play in two acts   
8                           saint hyacinth of poland   
9  rising sons and daughters life among japans ne...   

                                         description               authors  \
0                                            unknown          julie strain   
1  philip nel takes a fascinating look into the k...            philip nel   
2  this resource includes twelve principles in un...           david r ray   
3  julia thomas finds h

In [22]:
# Counting the 'unknown' label we created
unknown_summary = {}
for col in ['Title', 'description', 'authors', 'publisher', 'publishedDate', 'categories']:
    unknown_summary[col] = (df_amazon_books[col] == 'unknown').sum()

# Converting to DataFrame for a cleaner view
df_unknowns = pd.DataFrame.from_dict(unknown_summary, orient='index', columns=['Total Unknowns'])
df_unknowns['Percentage (%)'] = (df_unknowns['Total Unknowns'] / len(df_amazon_books)) * 100

print("--- Final Data Coverage Report ---")
print(df_unknowns.sort_values(by='Percentage (%)', ascending=False))

# Double check for any missed physical NaNs
print("\n--- Remaining Physical NaNs (Should be 0) ---")
print(df_amazon_books.isnull().sum())

--- Final Data Coverage Report ---
               Total Unknowns  Percentage (%)
publisher               75886       35.727199
description             68442       32.222557
categories              41199       19.396527
authors                 31413       14.789270
publishedDate           25305       11.913617
Title                       1        0.000471

--- Remaining Physical NaNs (Should be 0) ---
Title                 0
description           0
authors               0
image             52075
previewLink       23836
publisher             0
publishedDate         0
infoLink          23836
categories            0
ratingsCount     162652
dtype: int64


BOOKS Ratings CLEAN

In [23]:
# Load the reviews file
amazon_ratings_path = os.path.join(path_amazon, 'Books_rating.csv')
df_amazon_ratings = pd.read_csv(amazon_ratings_path)

# General Info: Row count and column types
print("--- 1. Data Types and Info ---")
print(df_amazon_ratings.info())

# Detailed Null Analysis
null_counts_rat = df_amazon_ratings.isnull().sum()
null_pct_rat = (null_counts_rat / len(df_amazon_ratings)) * 100
report_rat = pd.DataFrame({'Null Count': null_counts_rat, 'Percentage (%)': null_pct_rat})

print("\n--- 2. Null Values Report ---")
print(report_rat.sort_values(by='Percentage (%)', ascending=False))

# Statistical Summary (Scores and Prices)
print("\n--- 3. Statistical Summary ---")
print(df_amazon_ratings.describe())

# Preview of the main columns
print("\n--- 4. Data Sample (Title, Price, Score) ---")
# Adjusting column names based on typical Amazon dataset naming (Title, Price, review/score)
sample_cols = [col for col in ['Title', 'Price', 'review/score'] if col in df_amazon_ratings.columns]
print(df_amazon_ratings[sample_cols].head(10))

--- 1. Data Types and Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  object 
 1   Title               object 
 2   Price               float64
 3   User_id             object 
 4   profileName         object 
 5   review/helpfulness  object 
 6   review/score        float64
 7   review/time         int64  
 8   review/summary      object 
 9   review/text         object 
dtypes: float64(2), int64(1), object(7)
memory usage: 228.9+ MB
None

--- 2. Null Values Report ---
                    Null Count  Percentage (%)
Price                  2518829       83.960967
profileName             561905       18.730167
User_id                 561787       18.726233
review/summary             407        0.013567
Title                      208        0.006933
review/text                  8        0.000267
Id                           0 

In [24]:
import re

# Defining columns for text standardization
# Title is already cleaned; now we process the rest of the metadata
cols_to_standardize = ['Id', 'User_id', 'profileName', 'review/summary', 'review/text']

# Robust cleaning function (consistent with previous success)
def final_text_clean(text):
    if pd.isna(text) or str(text).lower() == 'nan':
        return 'unknown'
    text = str(text).lower()
    # Remove everything except letters, numbers, and spaces
    text = re.sub(r'[^a-z0-9\s]', '', text)
    # Collapse multiple spaces and trim edges
    return " ".join(text.split()).strip()

# Mass application to 3M rows
print("Standardizing columns... this is a heavy process, please wait.")
for col in cols_to_standardize:
    if col in df_amazon_ratings.columns:
        df_amazon_ratings[col] = df_amazon_ratings[col].apply(final_text_clean)
        print(f"Column '{col}' completed.")

# Quality check: Verification of clean strings and null removal
print("\n--- Final Standardization Verification ---")
print(df_amazon_ratings[cols_to_standardize + ['Title']].head(10))
print("\nPhysical Null Count (Should be 0 for these columns):")
print(df_amazon_ratings[cols_to_standardize].isnull().sum())

Standardizing columns... this is a heavy process, please wait.
Column 'Id' completed.
Column 'User_id' completed.
Column 'profileName' completed.
Column 'review/summary' completed.
Column 'review/text' completed.

--- Final Standardization Verification ---
           Id         User_id                      profileName  \
0  1882931173   avcgyzl8fqqtd                jim of oz jimofoz   
1  0826414346  a30tk6u7dns82r                    kevin killian   
2  0826414346  a3uh4uz4rsvo82                     john granger   
3  0826414346  a2mvuwt453qh61  roy e perry amateur philosopher   
4  0826414346  a22x4xupkf66mr      d h richards ninthwavestore   
5  0826414346  a2f6nonfudb6uk                           malvin   
6  0826414346  a14ojs0vwmoswo              midwest book review   
7  0826414346  a2rssxtdzdush4                         j squire   
8  0826414346  a25md5i2guiw6w            j p higbed big fellow   
9  0826414346  a3va4xfs5wnjo3                  donald burnside   

                

In [25]:
# Cleaning the Title column to match the metadata file
print("Normalizing Titles in the 3M dataset... please wait.")
df_amazon_ratings['Title'] = df_amazon_ratings['Title'].apply(final_text_clean)

# Final verification of the bridge column
print("\n--- Final Verification: Title Column ---")
print(df_amazon_ratings['Title'].head)

# Check for any remaining physical NaNs in Title
print(f"\nMissing Titles: {df_amazon_ratings['Title'].isnull().sum()}")

Normalizing Titles in the 3M dataset... please wait.

--- Final Verification: Title Column ---
<bound method NDFrame.head of 0          its only art if its well hung
1                 dr seuss american icon
2                 dr seuss american icon
3                 dr seuss american icon
4                 dr seuss american icon
                       ...              
2999995              the idea of history
2999996              the idea of history
2999997              the idea of history
2999998              the idea of history
2999999              the idea of history
Name: Title, Length: 3000000, dtype: object>

Missing Titles: 0


In [26]:
# Fill NaN prices using the maximum price found for each Title
# This ensures consistency across multiple reviews of the same book
print("Propagating prices by title... this might take a minute for 3M rows.")
df_amazon_ratings['Price'] = df_amazon_ratings['Price'].fillna(
    df_amazon_ratings.groupby('Title')['Price'].transform('max')
)

# Verification of the progress
null_after = df_amazon_ratings['Price'].isnull().sum()
pct_after = (null_after / len(df_amazon_ratings)) * 100

print(f"\n--- Price Rescue Report ---")
print(f"Remaining Nulls in Price: {null_after}")
print(f"New Null Percentage: {pct_after:.2f}%")

Propagating prices by title... this might take a minute for 3M rows.

--- Price Rescue Report ---
Remaining Nulls in Price: 2433951
New Null Percentage: 81.13%


In [27]:
# Performing the merge on the 'Title' column
# This will align categories and authors with their respective prices and scores
print("Merging datasets... this might take a moment due to the 3M rows.")
df_amazon_master = pd.merge(df_amazon_ratings, df_amazon_books, on='Title', how='inner')

# Checking the result of the union
print("\n--- Amazon Master Dataset: Initial Audit ---")
print(f"Total rows after merge: {len(df_amazon_master)}")
print(f"Columns available: {df_amazon_master.columns.tolist()}")

# Check for nulls in critical columns after merge
print("\n--- Nulls in Master Dataset ---")
print(df_amazon_master[['Title', 'Price', 'categories', 'authors']].isnull().sum())

Merging datasets... this might take a moment due to the 3M rows.

--- Amazon Master Dataset: Initial Audit ---
Total rows after merge: 3616827
Columns available: ['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness', 'review/score', 'review/time', 'review/summary', 'review/text', 'description', 'authors', 'image', 'previewLink', 'publisher', 'publishedDate', 'infoLink', 'categories', 'ratingsCount']

--- Nulls in Master Dataset ---
Title               0
Price         2977607
categories          0
authors             0
dtype: int64


In [28]:
# Calculate the average price per category and fill NaNs
# We use transform('mean') to broadcast the category average to all missing rows
print("Imputing prices by category... this will finalize our economic data.")
df_amazon_master['Price'] = df_amazon_master['Price'].fillna(
    df_amazon_master.groupby('categories')['Price'].transform('mean')
)

# If any nulls remain (categories with no prices at all), we use the global mean
global_mean = df_amazon_master['Price'].mean()
df_amazon_master['Price'] = df_amazon_master['Price'].fillna(global_mean)

# Final audit of the Price column
final_nulls = df_amazon_master['Price'].isnull().sum()
print(f"\n--- Final Price Audit ---")
print(f"Remaining Nulls in Price: {final_nulls}")
print(f"Final Average Price in Dataset: {df_amazon_master['Price'].mean():.2f}")

# Quick look at the enriched data
print("\n--- Amazon Master Sample ---")
print(df_amazon_master[['Title', 'categories', 'Price', 'review/score']].head(10))

Imputing prices by category... this will finalize our economic data.

--- Final Price Audit ---
Remaining Nulls in Price: 0
Final Average Price in Dataset: 21.93

--- Amazon Master Sample ---
                           Title               categories      Price  \
0  its only art if its well hung    comics graphic novels  13.184161   
1         dr seuss american icon  biography autobiography  18.583430   
2         dr seuss american icon  biography autobiography  18.583430   
3         dr seuss american icon  biography autobiography  18.583430   
4         dr seuss american icon  biography autobiography  18.583430   
5         dr seuss american icon  biography autobiography  18.583430   
6         dr seuss american icon  biography autobiography  18.583430   
7         dr seuss american icon  biography autobiography  18.583430   
8         dr seuss american icon  biography autobiography  18.583430   
9         dr seuss american icon  biography autobiography  18.583430   

   review/score

In [30]:
df_amazon_master.info()
df_amazon_master.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3616827 entries, 0 to 3616826
Data columns (total 19 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  object 
 1   Title               object 
 2   Price               float64
 3   User_id             object 
 4   profileName         object 
 5   review/helpfulness  object 
 6   review/score        float64
 7   review/time         int64  
 8   review/summary      object 
 9   review/text         object 
 10  description         object 
 11  authors             object 
 12  image               object 
 13  previewLink         object 
 14  publisher           object 
 15  publishedDate       object 
 16  infoLink            object 
 17  categories          object 
 18  ratingsCount        float64
dtypes: float64(3), int64(1), object(15)
memory usage: 524.3+ MB


,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,1882931173,its only art if its well hung,13.184161,avcgyzl8fqqtd,jim of oz jimofoz,7/7,4.0,940636800,nice collection of julie strain images,this is only for julie strain fans its a colle...,unknown,julie strain,http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,unknown,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,comics graphic novels,NaN
1,0826414346,dr seuss american icon,18.583430,a30tk6u7dns82r,kevin killian,10/10,5.0,1095724800,really enjoyed it,i dont care much for dr seuss but after readin...,philip nel takes a fascinating look into the k...,philip nel,http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,ac black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,biography autobiography,NaN
2,0826414346,dr seuss american icon,18.583430,a3uh4uz4rsvo82,john granger,10/11,5.0,1078790400,essential for every personal and public library,if people become the books they read and if th...,philip nel takes a fascinating look into the k...,philip nel,http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,ac black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,biography autobiography,NaN
3,0826414346,dr seuss american icon,18.583430,a2mvuwt453qh61,roy e perry amateur philosopher,7/7,4.0,1090713600,phlip nel gives silly seuss a serious treatment,theodore seuss geisel 19041991 aka quotdr seus...,philip nel takes a fascinating look into the k...,philip nel,http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,ac black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,biography autobiography,NaN
4,0826414346,dr seuss american icon,18.583430,a22x4xupkf66mr,d h richards ninthwavestore,3/3,4.0,1107993600,good academic overview,philip nel dr seuss american iconthis is basic...,philip nel takes a fascinating look into the k...,philip nel,http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,ac black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,biography autobiography,NaN


DF MASTER AMAZON BOOKS

In [31]:
import pandas as pd

# 1. List of selected columns
selected_columns = [
    'Id', 'Title', 'Price', 'User_id', 'review/score', 
    'authors', 'publisher', 'publishedDate', 'categories', 'ratingsCount'
]

# 2. Apply filter to the main DataFrame
df_amazon_master = df_amazon_master[selected_columns]

# 3. Rename columns for a more professional look (snake_case)
# This step is optional but highly recommended for data analysts
df_amazon_master.columns = [
    'book_id', 'title', 'price', 'user_id', 'rating', 
    'authors', 'publisher', 'published_date', 'categories', 'ratings_count'
]

# 4. Final Verification
print("--- Final Dataset Information ---")
print(df_amazon_master.info())

# 5. Export to CSV
df_amazon_master.to_csv('Master_AmazonBooks_Data.csv', index=False, encoding='utf-8')
print("\nExport completed: Master_AmazonBooks_Data.csv")

--- Final Dataset Information ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3616827 entries, 0 to 3616826
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   book_id         object 
 1   title           object 
 2   price           float64
 3   user_id         object 
 4   rating          float64
 5   authors         object 
 6   publisher       object 
 7   published_date  object 
 8   categories      object 
 9   ratings_count   float64
dtypes: float64(3), object(7)
memory usage: 275.9+ MB
None

Export completed: Master_AmazonBooks_Data.csv


GOODREADS BOOKS CLEAN 

In [29]:
import pandas as pd
import os
import re

# Load Goodreads books with the correct syntax
goodreads_books_path = os.path.join(path_goodreads, 'books.csv')
df_goodreads = pd.read_csv(goodreads_books_path, on_bad_lines='skip')

# Initial Null Audit
print("--- Initial Null Analysis (Goodreads) ---")
print(df_goodreads.isnull().sum())

# Handling Nulls before standardization
# For text columns, we use 'unknown'. For numeric ratings, we'll keep them as is for now.
df_goodreads['title'] = df_goodreads['title'].fillna('unknown')
df_goodreads['authors'] = df_goodreads['authors'].fillna('unknown')
df_goodreads['publisher'] = df_goodreads['publisher'].fillna('unknown')

# Aggressive Standardization Function
def gr_deep_clean(text):
    if pd.isna(text) or str(text).lower() == 'nan':
        return 'unknown'
    text = str(text).lower()
    # Keep only letters, numbers and spaces
    text = re.sub(r'[^a-z0-9\s]', '', text)
    # Remove extra spaces
    return " ".join(text.split()).strip()

# Applying standardization to key columns
cols_to_fix = ['title', 'authors', 'publisher']
for col in cols_to_fix:
    df_goodreads[col] = df_goodreads[col].apply(gr_deep_clean)

# Global renaming for future matching
df_goodreads = df_goodreads.rename(columns={'title': 'Title'})

# Final Verification of Goodreads Data Quality
print("\n--- Final Quality Report: Goodreads ---")
print(df_goodreads[['Title', 'authors', 'average_rating']].head(10))
print("\nRemaining physical nulls in text columns:")
print(df_goodreads[['Title', 'authors', 'publisher']].isnull().sum())

--- Initial Null Analysis (Goodreads) ---
bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
  num_pages           0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64

--- Final Quality Report: Goodreads ---
                                               Title                 authors  \
0  harry potter and the halfblood prince harry po...  jk rowlingmary grandpr   
1  harry potter and the order of the phoenix harr...  jk rowlingmary grandpr   
2  harry potter and the chamber of secrets harry ...              jk rowling   
3  harry potter and the prisoner of azkaban harry...  jk rowlingmary grandpr   
4    harry potter boxed set books 15 harry potter 15  jk rowlingmary grandpr   
5  unauthorized harry potter book seven news half...   w frederick zimmerman   
6            harry potter collection harry potter 16    

DF GOODREADS 

In [32]:
## Exporting Cleaned Goodreads Data
import pandas as pd

# 1. Define the output filename
output_file = 'Master_GoodreadsBooks_Data.csv'

# 2. Export the DataFrame
# We use index=False to keep the CSV clean
# We use encoding='utf-8' to preserve any special characters after the regex clean
df_goodreads.to_csv(output_file, index=False, encoding='utf-8')

print(f"Success: {output_file} has been saved with cleaned and standardized data.")

Success: Master_GoodreadsBooks_Data.csv has been saved with cleaned and standardized data.
